In [2]:
import sys
sys.path.append("..")   # tell Python to also look one folder up

from src.dcf import *
from src.wacc import *
from src.comps import *
from src.utils import *

print("Imports loaded successfully.")


Imports loaded successfully.


In [3]:
!pip install -r ../requirements.txt

In [4]:
import numpy as np
import pandas as pd

# --- Simple toy assumptions (placeholder numbers) ---

years = np.arange(1, 6)  # 5-year forecast
fcf = np.array([18, 19, 20, 21, 22])  # $bn, fake numbers for demo
wacc = 0.08                     # 8% discount rate (placeholder)
g_terminal = 0.02               # 2% terminal growth

# --- Build the forecast table ---

df = pd.DataFrame({
    "Year": years,
    "FCF ($bn)": fcf
})

df

,Year,FCF ($bn)
0,1,18
1,2,19
2,3,20
3,4,21
4,5,22


In [5]:
# Discount factors
df["Discount Factor"] = 1 / (1 + wacc) ** df["Year"]
df["PV of FCF ($bn)"] = df["FCF ($bn)"] * df["Discount Factor"]

# Terminal value at the end of Year 5 (Gordon growth)
fcf_5 = df.loc[df["Year"] == 5, "FCF ($bn)"].iloc[0]
terminal_value = fcf_5 * (1 + g_terminal) / (wacc - g_terminal)

# Present value of terminal value
df.loc[df["Year"] == 5, "PV of TV ($bn)"] = terminal_value * df.loc[df["Year"] == 5, "Discount Factor"]

enterprise_value = df["PV of FCF ($bn)"].sum() + df["PV of TV ($bn)"].fillna(0).sum()

print("WACC:          {:.2%}".format(wacc))
print("Terminal g:    {:.2%}".format(g_terminal))
print("Enterprise EV: ${:,.1f} bn".format(enterprise_value))

df

WACC:          8.00%
Terminal g:    2.00%
Enterprise EV: $333.8 bn


,Year,FCF ($bn),Discount Factor,PV of FCF ($bn),PV of TV ($bn)
0,1,18,0.925926,16.666667,NaN
1,2,19,0.857339,16.289438,NaN
2,3,20,0.793832,15.876645,NaN
3,4,21,0.735030,15.435627,NaN
4,5,22,0.680583,14.972830,254.538116


In [6]:
df.to_csv("../data/processed/exxon_dcf_base_case.csv", index=False)
print("Saved to data/processed/exxon_dcf_base_case.csv")

Saved to data/processed/exxon_dcf_base_case.csv


In [7]:
import numpy as np
import pandas as pd

def dcf_enterprise_value(fcf_array, wacc, g_terminal):
    """
    Simple DCF engine using the same logic as above:
    - discount explicit FCFs
    - add Gordon-growth terminal value in final year
    - return EV (no net debt handled here for now)
    """
    years = np.arange(1, len(fcf_array) + 1)

    df = pd.DataFrame({
        "Year": years,
        "FCF ($bn)": fcf_array
    })

    df["Discount Factor"] = 1 / (1 + wacc) ** df["Year"]
    df["PV of FCF ($bn)"] = df["FCF ($bn)"] * df["Discount Factor"]

    # Terminal value (Gordon growth)
    fcf_last = df.loc[df["Year"] == years[-1], "FCF ($bn)"].iloc[0]
    terminal_value = fcf_last * (1 + g_terminal) / (wacc - g_terminal)

    # PV of terminal value in final year
    df.loc[df["Year"] == years[-1], "PV of TV ($bn)"] = terminal_value * df.loc[df["Year"] == years[-1], "Discount Factor"]

    enterprise_value = df["PV of FCF ($bn)"].sum() + df["PV of TV ($bn)"].fillna(0).sum()

    return enterprise_value, df


## Scenario Analysis - Base, Bull, and Bear Cases (Stage-2 Demo)

This section illustrates how valuation outcomes shift under alternative
assumptions for ExxonMobil’s free cash flow trajectory, discount rate
(WACC) and terminal growth. The scenarios use placeholder figures for
demonstration purposes only (i.e., not real ExxonMobil data yet):

- **Base Case:** central view of operating performance: steady FCF
  progression, midpoint WACC, neutral terminal growth

- **Bull Case:** +10% uplift in free cash flow, slightly lower WACC
  reflecting improved risk perception, and a higher terminal growth rate

- **Bear Case:** -10% reduction in free cash flow, slightly higher WACC
  due to increased perceived risk, and a more conservative terminal
  growth assumption

This structure mirrors the scenario frameworks used in equity research
and investment banking models. In **Stage 3**, the same logic will be
populated with actual ExxonMobil financials and market-derived cost of
capital inputs.

In [8]:
# Base toy FCFs from your earlier example
base_fcf = np.array([18, 19, 20, 21, 22])  # $bn, demo

scenarios = {
    "Base": {
        "fcf": base_fcf,
        "wacc": 0.08,
        "g": 0.02
    },
    "Bull": {
        "fcf": base_fcf * 1.10,   # +10% FCF
        "wacc": 0.075,            # slightly lower WACC
        "g": 0.022                # slightly higher terminal g
    },
    "Bear": {
        "fcf": base_fcf * 0.90,   # -10% FCF
        "wacc": 0.085,            # slightly higher WACC
        "g": 0.018                # slightly lower g
    }
}

rows = []

for name, params in scenarios.items():
    ev, _df = dcf_enterprise_value(
        fcf_array=params["fcf"],
        wacc=params["wacc"],
        g_terminal=params["g"]
    )
    rows.append({
        "Scenario": name,
        "WACC": params["wacc"],
        "Terminal g": params["g"],
        "EV ($bn, demo)": round(ev, 1)
    })

scen_df = pd.DataFrame(rows)
scen_df

,Scenario,WACC,Terminal g,"EV ($bn, demo)"
0,Base,0.080,0.020,333.8
1,Bull,0.075,0.022,413.4
2,Bear,0.085,0.018,270.4


In [9]:
wacc_range = np.linspace(0.075, 0.085, 5)      # 7.5% to 8.5%
g_range = np.linspace(0.018, 0.022, 5)         # 1.8% to 2.2%

sens_rows = []

for w in wacc_range:
    for g in g_range:
        ev, _ = dcf_enterprise_value(base_fcf, wacc=w, g_terminal=g)
        sens_rows.append({
            "WACC": round(w, 4),
            "g": round(g, 4),
            "EV ($bn, demo)": round(ev, 1)
        })

sens_df = pd.DataFrame(sens_rows)
sens_table = sens_df.pivot(index="g", columns="WACC", values="EV ($bn, demo)")
sens_table

WACC,0.0750,0.0775,0.0800,0.0825,0.0850
g,,,,,
0.018,354.0,338.9,325.1,312.3,300.5
0.019,359.2,343.6,329.4,316.2,304.1
0.020,364.5,348.5,333.8,320.3,307.8
0.021,370.1,353.5,338.3,324.4,311.6
0.022,375.8,358.7,343.1,328.7,315.5


## Sensitivity Analysis - WACC vs Terminal Growth

This section evaluates how ExxonMobil’s enterprise value responds to
changes in two key valuation drivers:

- **WACC (Weighted Average Cost of Capital):** the discount rate applied
  to future cash flows  
- **Terminal Growth Rate (g):** the long-run growth assumption used in
  the terminal value calculation

The table below shows a valuation grid where each cell represents the
enterprise value implied by a different combination of WACC and terminal
growth. This mirrors the sensitivity tables typically included in
investment banking DCF outputs, allowing users to assess valuation
resilience under alternative macro and capital-market assumptions.

## Valuation Summary (Stage-2 Demo)

This stage of the model demonstrates the valuation mechanics using illustrative inputs. At this stage, numbers are placeholders. In the next stage, the same framework will be populated with real ExxonMobil financials and market-derived WACC inputs.

## Current Status: 
- **Stage 2:** Model logic validated using demo inputs. 
- **Stage 3:** Load real ExxonMobil financials and market WACC.